In [1]:
%%capture

# Install/Upgrade Latest Version of TopGun toolkit
!python -m pip install git+https://github.com/djmcnay/topgun#egg=topgun --upgrade
import topgun

IndentationError: unexpected indent (backtest.py, line 217)

In [5]:
import numpy as np
import pandas as pd
import xlwings as xlw

wb = xlw.Book('BACKTEST.xlsm')

# index data from timeseries sheet; CONVERT TO RETURNS & make sure date strings are datetimes
bmkrtns = wb.sheets['TIMESERIES'].range('D1').options(pd.DataFrame, expand='table').value.iloc[3:,:].pct_change()
bmkrtns.index = pd.to_datetime(bmkrtns.index)
display(bmkrtns.tail())    # preview

# Bring in Enhanced & Core Data
E = wb.sheets['Enhanced'].range('A1').options(pd.DataFrame, expand='table').value.iloc[:,1]
C = wb.sheets['Core'].range('A1').options(pd.DataFrame, expand='table').value.iloc[:,1]
SMM = wb.sheets['SMM'].range('A1').options(pd.DataFrame, expand='table').value.iloc[:,0]


# Convert to month-end dates
E.index = E.index + pd.offsets.MonthEnd(0)
C.index = C.index + pd.offsets.MonthEnd(0)
SMM.index = pd.to_datetime(SMM.index) + pd.offsets.MonthEnd(0)

# Rename Series
E.name = 'Enhanced'
C.name = 'Core'
SMM.name ='SMM'

benchmark = 'SWIX'    # Becnhmark code from benchmarks

,ZAR,JALSH,SWIX,SAGB,STEFI,MXWO,MXEF,GBL_AGG
Dates,,,,,,,,
2020-06-30,-0.009546,0.077416,0.069813,0.002584,0.004704,0.014975,0.061507,0.004991
2020-07-31,-0.017677,0.025563,0.030073,0.025390,0.004189,0.026455,0.067127,0.010928
2020-08-31,-0.008376,-0.002575,-0.008797,0.014494,0.003874,0.060829,0.016372,-0.007246
2020-09-30,-0.014337,-0.015847,-0.010665,0.014404,0.003486,-0.048341,-0.030146,0.003653
2020-10-30,-0.026501,-0.047255,-0.042437,0.035083,0.003269,-0.054533,-0.004517,0.000073


In [3]:
bt = topgun.BacktestAnalytics(E, bmkrtns, benchmark=benchmark, Rf='STEFI')
md = bt.big_bang(title="MF Enhanced Strategy - Fairtree UT")
topgun.Reporting().md2html(md=md, title="MF Enhanced v2")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning:

invalid value encountered in double_scalars



'REPORT WRITTEN: MF Enhanced v1'

In [7]:
# PORT(s) to Model
rtns = pd.concat([E, C], axis=1).dropna()
rtns['E20'] = rtns['Enhanced'] * 0.2 + rtns['Core'] * (1 - 0.2)
rtns['E30'] = rtns['Enhanced'] * 0.3 + rtns['Core'] * (1 - 0.3)
rtns['E40'] = rtns['Enhanced'] * 0.4 + rtns['Core'] * (1 - 0.4)
rtns['E50'] = rtns['Enhanced'] * 0.5 + rtns['Core'] * (1 - 0.5)
rtns['E60'] = rtns['Enhanced'] * 0.6 + rtns['Core'] * (1 - 0.6)
rtns['E70'] = rtns['Enhanced'] * 0.7 + rtns['Core'] * (1 - 0.7)
rtns['E80'] = rtns['Enhanced'] * 0.8 + rtns['Core'] * (1 - 0.8)
rtns = rtns.drop(['Enhanced', 'Core'], axis=1)

bt = topgun.BacktestAnalytics(rtns, bmkrtns, benchmark=benchmark, Rf='STEFI')
md = bt.big_bang(title="Multi-Factor Enhanced-Core Blends")
topgun.Reporting().md2html(md=md, title="MF Blends v2")

'REPORT WRITTEN: MF Blends v2'

In [6]:
# PORT(s) to Model
rtns = pd.concat([E, C], axis=1).dropna()
ewgt = 0.3
rtns['MIX'] = rtns['Enhanced']*ewgt + rtns['Core']*(1-ewgt)

bt = topgun.BacktestAnalytics(rtns, bmkrtns, benchmark=benchmark, Rf='STEFI')
md = bt.big_bang(title="Multi-Factor Historical Returns with {:.0%} Blend".format(ewgt))
topgun.Reporting().md2html(md=md, title="MF Historical v2")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning:

invalid value encountered in double_scalars

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning:

invalid value encountered in less



'REPORT WRITTEN: MF Historical v2'

In [6]:
rtns = pd.concat([E, C, SMM], axis=1).dropna()
rtns['MF'] = rtns['Enhanced']*0.30 + rtns['Core']*(1-0.3)
rtns['BLEND'] = rtns['MF']*0.5 + rtns['SMM']*(1-0.5)

rtns = rtns.loc[:,['BLEND', 'SMM', 'MF', 'Enhanced', 'Core']]

bt = topgun.BacktestAnalytics(rtns, bmkrtns, benchmark=benchmark, Rf='STEFI')
md = bt.big_bang(title="SMM Equity Fund")
topgun.Reporting().md2html(md=md, title="SMM Equity Blend v2")

'REPORT WRITTEN: SMM Equity Blend v1'